In [2]:
import sys
import copy
import time
from urllib.parse import unquote
import requests
from urllib.parse import quote
import re
from lxml import etree

## Login

In [3]:
def login(laccount, lpassword):
    """ 根据账号密码登录linkedin """
    s = requests.Session()
    r = s.get('https://www.linkedin.com/uas/login')
    tree = etree.HTML(r.content)
    loginCsrfParam = ''.join(tree.xpath('//input[@id="loginCsrfParam-login"]/@value'))
    csrfToken = ''.join(tree.xpath('//input[@id="csrfToken-login"]/@value'))
    sourceAlias = ''.join(tree.xpath('//input[@id="sourceAlias-login"]/@value'))
    isJsEnabled = ''.join(tree.xpath('//input[@name="isJsEnabled"]/@value'))
    source_app = ''.join(tree.xpath('//input[@name="source_app"]/@value'))
    tryCount = ''.join(tree.xpath('//input[@id="tryCount"]/@value'))
    clickedSuggestion = ''.join(tree.xpath('//input[@id="clickedSuggestion"]/@value'))
    signin = ''.join(tree.xpath('//input[@name="signin"]/@value'))
    session_redirect = ''.join(tree.xpath('//input[@name="session_redirect"]/@value'))
    trk = ''.join(tree.xpath('//input[@name="trk"]/@value'))
    fromEmail = ''.join(tree.xpath('//input[@name="fromEmail"]/@value'))

    payload = {
        'isJsEnabled': isJsEnabled,
        'source_app': source_app,
        'tryCount': tryCount,
        'clickedSuggestion': clickedSuggestion,
        'session_key': laccount,
        'session_password': lpassword,
        'signin': signin,
        'session_redirect': session_redirect,
        'trk': trk,
        'loginCsrfParam': loginCsrfParam,
        'fromEmail': fromEmail,
        'csrfToken': csrfToken,
        'sourceAlias': sourceAlias
    }
    s.post('https://www.linkedin.com/uas/login-submit', data=payload)
    return s

In [4]:
s = login(laccount='tsb1071463@gmail.com', lpassword='Alisa0504')

In [118]:
company_name = input('Input the company you want to crawl:')

Input the company you want to crawl:台新


In [89]:
maxpage = 50 

## get_Linkedin_url

In [119]:
url = 'https://www.google.com/search?q=Linkedin+' + quote(company_name)+'&oq=Linkedin+'+ quote(company_name)

In [120]:
res = requests.get(url,timeout=10)

In [121]:
from bs4 import BeautifulSoup as bs
soup = bs(res.text,"lxml")

In [122]:
from pprint import pprint

In [126]:
[(i.text,i.parent['href'].split('q=')[1].split('&')[0]) for i in soup.select('a > div.BNeawe.vvjwJb.AP7Wnd')]

[('Taishin International Commercial Bank | LinkedIn',
  'https://www.linkedin.com/company/%25E5%258F%25B0%25E6%2596%25B0%25E5%259C%258B%25E9%259A%259B%25E5%2595%2586%25E6%25A5%25AD%25E9%258A%2580%25E8%25A1%258C%25E8%2582%25A1%25E4%25BB%25BD%25E6%259C%2589%25E9%2599%2590%25E5%2585%25AC%25E5%258F%25B8'),
 ('Taishin Financial Holdings | LinkedIn',
  'https://www.linkedin.com/company/taishin-international-bank'),
 ('Julia Kan - Project Manager - 台新金控| LinkedIn',
  'https://tw.linkedin.com/in/julia-kan-13671612b'),
 ('Anny Lee - 經理- 台新銀行| LinkedIn',
  'https://tw.linkedin.com/in/anny-lee-a630aa163'),
 ('趙毓齡- 經理- 台新國際商業銀行股份有限公司| LinkedIn',
  'https://tw.linkedin.com/in/%25E6%25AF%2593%25E9%25BD%25A1-%25E8%25B6%2599-725ba3118'),
 ('Dickson Fong - AVP compliance - 台新國際商業銀行| LinkedIn',
  'https://hk.linkedin.com/in/dickson-fong-710064170'),
 ('LC AB 台新銀行郭立程- VP - Taishin International Bank | LinkedIn',
  'https://hk.linkedin.com/in/lc-ab-%25E5%258F%25B0%25E6%2596%25B0%25E9%258A%2580%25E8%25A1%2

In [ ]:
results = []
failure = 0
while len(url) > 0 and failure < 10:
    try:
        r = requests.get(url, timeout=10)
    except Exception as e:
        failure += 1
        continue
    if r.status_code == 200:
        hrefs=[i.parent['href'].split('q=')[1].split('&')[0] for i in soup.select('a > div.BNeawe.vvjwJb.AP7Wnd')]

In [ ]:
results = []
failure = 0
while len(url) > 0 and failure < 10:
    try:
        r = requests.get(url, timeout=10)
    except Exception as e:
        failure += 1
        continue
    if r.status_code == 200:
#         hrefs = list(set(re.findall('"(http://www\.baidu\.com/link\?url=.*?)"', r.text))) 
        hrefs = [i['href'] for i in soup.select('a')]    # 一页有10个搜索结果
        for href in hrefs:
            crawl(href, copy.deepcopy(s))
        results += hrefs
        tree = etree.HTML(r.content)
        nextpage_txt = tree.xpath('//div[@id="page"]/a[@class="n" and contains(text(), "下一页")]/@href'.decode('utf8'))
        url = 'http://www.baidu.com' + nextpage_txt[0].strip() if nextpage_txt else ''
        failure = 0
        maxpage -= 1
        if maxpage <= 0:
            break
    else:
        failure += 2
        print ('search failed: %s' % r.status_code)
if failure >= 10:
    print ('search failed: %s' % url)

In [139]:
def get_linkedin_url(url, s):
    try:
        r = s.get(url, allow_redirects=False)
        if r.status_code == 999 and 'Location' in r.headers.keys() and 'linkedin.com/in/' in r.headers['Location']:
            return r.headers['Location']
    except Exception as e:
        print ('get linkedin url failed: %s' % url)
    return ''

In [134]:
hrefs=[i.parent['href'].split('q=')[1].split('&')[0] for i in soup.select('a > div.BNeawe.vvjwJb.AP7Wnd')]

In [171]:
hrefs[2]

'https://tw.linkedin.com/in/julia-kan-13671612b'

In [179]:
re.findall(r'^https://tw\.linkedin\.com/in/',hrefs[2])

['https://tw.linkedin.com/in/']

In [141]:
rr = copy.deepcopy(s).get(hrefs[0],allow_redirects=False)

In [142]:
rr.status_code

999

In [147]:
rr.headers.keys()

KeysView({'Date': 'Wed, 07 Aug 2019 09:27:24 GMT', 'X-Li-Pop': 'prod-ehk1', 'X-LI-Proto': 'http/1.1', 'X-LI-UUID': 'GmxU69yZuBVgWt5ffisAAA==', 'Set-Cookie': 'trkCode=ripf; Max-Age=5, trkInfo=AQHCIQJy9t_lagAAAWxrZ4xgHClzlcLg4OgzrKTS9wyIzildH-SyCjbU3hnB2uJTXfc9oAPVV-HWdpWWf4laoUartcW4LviILqxw3TwFuXJVGBRcMlLHtlNe1m5TLANSZV1S4lU=; Max-Age=5, rtc=AQFgSEmiBcZv_wAAAWxrZ4xgsss-o4oDraVJ5ZI7KwkT1DEidZWVUDSOV-rap6gM_E25_Z6s3AnFw_7wJRq9vX5SFj8I1yAPnB06WamHBsM8XRvfB6vAczcYB63qwl8jW7AC7Dz9-Vt18Nfi9mioXT6Q6nsswreERNBQ8J-BtV8Eh4e3V33JNuRHT4rDumeKgfJoEJ65h8z4R7ILSUFj-SEFtHoSM8CMavVc7tj9ZxFBXNenwS4RBy5-ExWK67qpFg110F7NwR_s6tj-4gloPopivgAg9BCznjCykw==; Max-Age=120; path=/; domain=.linkedin.com', 'Content-Length': '1461', 'Content-Type': 'text/html'})

In [140]:
for href in hrefs:
    try:
        url = get_linkedin_url(url, copy.deepcopy(s))
        if len(url) > 0 and url not in LINKS_FINISHED:
            LINKS_FINISHED.append(url)
    except Exception as e:
        pass

get linkedin url failed: 
get linkedin url failed: 
get linkedin url failed: 
get linkedin url failed: 
get linkedin url failed: 
get linkedin url failed: 
get linkedin url failed: 
get linkedin url failed: 
get linkedin url failed: 
get linkedin url failed: 


In [ ]:
def crawl(url, s):
    """ 抓取每一个搜索结果 """
    try:
        url = get_linkedin_url(url, copy.deepcopy(s)).replace('cn.linkedin.com', 'www.linkedin.com')  
        if len(url) > 0 and url not in LINKS_FINISHED:
            LINKS_FINISHED.append(url)

            failure = 0
            while failure < 10:
                try:
                    r = s.get(url, timeout=10)
                except Exception as e:
                    failure += 1
                    continue
                if r.status_code == 200:
                    parse(r.content, url)
                    break
                else:
                    print ('%s %s' % (r.status_code, url))
                    failure += 2
            if failure >= 10:
                print ('Failed: %s' % url)
    except Exception as e:
        pass
